## Pandas Practice

In [ ]:
# install required libs

#!pip install azure.storage.blob
#!pip install polars

     -------------------------------------- 26.8/26.8 MB 387.7 kB/s eta 0:00:00


In [32]:
#import required libraries 
import pandas as pd
from azure_hepler_functions import blob_to_df
import json

In [10]:
config_file_path = '../config.json'
with open(config_file_path, 'r') as f:
    config_data = json.load(f)
connection_string = config_data['connection_string']

In [11]:
SalesDF = blob_to_df(connection_string,
           "sales", "sales.csv")

d:\dev\data engineering with python\python-for-de-practice\azure_hepler_functions.py:122: DtypeWarning: Columns (9,11,14,17,18,20,21,28,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(io.StringIO(blob_data_stream))


In [12]:
'Total rows: '+ str(SalesDF.shape[0])+'    Total columns: '+ str(SalesDF.shape[1])

'Total rows: 1713136    Total columns: 47'

In [13]:
SalesDF.shape

(1713136, 47)

In [14]:
#Loading all the dimension tables
LocationDF = pd.read_csv('dim_location.csv')
ChannelDF = pd.read_csv('dim_channel.csv')
CustomerDF = pd.read_csv('dim_customer.csv')
MenuDF = pd.read_csv('dim_menu.csv')
ItemDF = pd.read_csv('dim_item.csv')

In [15]:
#SalesDF.columns

In [16]:
# Drop unwanted columns from the fact table
columns_to_drop = ['Price Group Code', 'Periodic Disc_ Type', 'Periodic Disc_ Group', 'Total GST Amount', 'DayPart', 
                   'DayPart close time', 'dummy all', 'Hour', 'Hour end', 'minutes in ending hour', 'minutes in starting hour', 
                   'Time diff', 'Visit number 2', 'Manual Post', 'MPOS ID', 'Dotpe Aggregator Order', 'key2', 'NSO week 2', 
                   'Sales Staff', 'Net_Amount', 'e-Commerce Customer', 'e-Commerce Merchant Id', 'Event Source', 'Dotpe Order',
                   'No_ of Covers']

SalesDF.drop(columns=columns_to_drop, inplace=True)

In [17]:
# rename columns
new_column_names = {'Store No_':'store_number',
                   'Receipt No_':'receipt_number',
                   'Item No_':'item_number',
                   'Net Amount':'net_amount',
                   'Customer No_':'customer_number',
                   'Loylty Item':'loyalty_item',
                   'Deal description temp':'deal_description',
                   'Deal ideal price':'deal_ideal_price',
                   'Deal no':'deal_number',
                   'Offer no':'offer_number',
                   'Gross amount':'gross_amount',
                   'No_ of Items':'total_items',
                   'Customer Order No_':'customer_order_number',
                   'Sell-to Contact No_':'contact',
                   'Quantity':'quantity',
                   'Date':'date',
                   'Cost Amount':'cost_amount',
                   'Discount Amount':'discount',
                   'Total Discount':'total_discount',
                   'Offer Type':'offer_type',
                   'Sales Type':'sales_type'}

SalesDF.rename(columns=new_column_names, inplace=True)

In [18]:
# change date column's datatype 
SalesDF['date'] = SalesDF['date'].astype('datetime64') 

In [19]:
#change quantity to absolute value and change it to integer
SalesDF['quantity'] = SalesDF['quantity'].apply(lambda x: int(abs(x)))

#change net_amount, cost_amount & gross_amount to absolute value
SalesDF['net_amount'] = SalesDF['net_amount'].apply(abs)
SalesDF['cost_amount'] = SalesDF['cost_amount'].apply(abs)
SalesDF['gross_amount'] = SalesDF['gross_amount'].apply(abs)

In [20]:
# SalesDF
# LocationDF
# ChannelDF
# CustomerDF
# MenuDF
# ItemDF

In [21]:
#Join sales table with loaction and menu tables

JoinedDF = pd.merge(
                    pd.merge(
                        SalesDF, 
                        LocationDF[['Loc Code', 'As per ERP']]
                        .rename(columns={'As per ERP':'store_location', 'Loc Code':'store_number'}), 
                        left_on='store_number', 
                        right_on='store_number', 
                        how='left'),
                    MenuDF[['Actual Item Code', 'Item Name', 'Sub-Category', 'Category']]
                    .rename(columns={'Actual Item Code':'item_number',
                                     'Item Name':'item_name',
                                     'Sub-Category':'item_sub_category',
                                     'Category':'item_category'}),
                    on='item_number',
                    how='left')
                

In [22]:
#JoinedDF.columns

In [23]:
JoinedDF

,store_number,receipt_number,item_number,quantity,date,net_amount,cost_amount,discount,customer_number,total_discount,...,gross_amount,offer_type,total_items,sales_type,customer_order_number,contact,store_location,item_name,item_sub_category,item_category
0,N1103,00000MGRTA000015778,1GGT0336,2,2024-01-01,9.52,7.900,0.00,ZMTHR,0.0,...,9.52,14.0,5.0,DELIVERY,5464420749,NaN,MG Road Cloud,Addon Onion and Lime,Addon Onion/lime,Accompaniments
1,N1103,00000MGRTA000015778,1GGT0196,2,2024-01-01,393.00,168.876,125.00,ZMTHR,0.0,...,393.00,14.0,5.0,DELIVERY,5464420749,NaN,MG Road Cloud,Royal Chicken Biryani,Royal,Biryani
2,N1103,00000MGRTA000015778,PAKCRG,1,2024-01-01,23.81,23.810,0.00,ZMTHR,0.0,...,23.81,14.0,5.0,DELIVERY,5464420749,NaN,MG Road Cloud,Packaging Charges,Carry Bag,Packaging
3,N1103,00000MGRTA000015707,1GGT0196,1,2024-01-01,184.00,84.438,75.00,SWGHR,0.0,...,184.00,14.0,2.0,DELIVERY,163071193404565,NaN,MG Road Cloud,Royal Chicken Biryani,Royal,Biryani
4,N1103,00000MGRTA000015707,PAKCRG,1,2024-01-01,23.81,23.810,0.00,SWGHR,0.0,...,23.81,14.0,2.0,DELIVERY,163071193404565,NaN,MG Road Cloud,Packaging Charges,Carry Bag,Packaging
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713131,S6107,00TNGTB000156222,2BV10069,1,2024-01-31,36.20,17.640,0.00,NaN,0.0,...,36.20,NaN,1.0,TAKEAWAY,NaN,NaN,T Nagar Express,Coke Pet 600ML,Soft Drinks,Beverages
1713132,S6107,00TNGTB000156226,1CHUP079,1,2024-01-31,358.00,116.090,0.00,ZMTTN,0.0,...,358.00,14.0,4.0,DELIVERY,5526051355,NaN,T Nagar Express,Kebab Roll- Chicken Tikka ( 2 no’s ),Rolls,Snackers
1713133,S6107,00TNGTB000156226,PAKCRG,1,2024-01-31,23.81,23.810,0.00,ZMTTN,0.0,...,23.81,14.0,4.0,DELIVERY,5526051355,NaN,T Nagar Express,Packaging Charges,Carry Bag,Packaging
1713134,S6107,00TNGTB000156226,1CHT0214,1,2024-01-31,204.65,120.048,64.35,ZMTTN,0.0,...,204.65,14.0,4.0,DELIVERY,5526051355,NaN,T Nagar Express,Haleem,Haleem,Haleem


# Window Function like transformations

## Python window function catagories

 ### Aggregate: group, rolling, expand
 ### Ranking: row numbers, rank, dense rank, percent, n-tile/qcut()
 ### Value: lead/lag, first/last/nth value

In [24]:
# simple groupby
# date-wise total sales

date_wise_sales = JoinedDF.groupby(['date', 'store_location']).agg({'net_amount': ['sum', 'mean']}) #for moltiple agg on one col
date_wise_sales = JoinedDF.groupby(['date', 'store_location']).agg({'net_amount': 'sum'}) #for simple agg
date_wise_sales = JoinedDF.groupby(['date', 'store_location']).agg({'net_amount': 'sum', 'quantity': 'mean'})
#date_wise_sales

In [25]:
date_wise_sales

net_amount  quantity
date       store_location                          
2024-01-01 Attapur               47745.76  1.475248
           BHEL                 237161.06  1.669866
           Batasingaram          26919.01  1.643564
           Battarahalli Cloud    39206.02  1.040724
           Begumpet             292166.20  1.843893
...                                   ...       ...
2024-01-31 VIP Road              83056.21  1.680129
           Vanasthalipuram       48804.34  1.659280
           Vizag Beach Road      28141.22  1.556034
           Warangal              55543.15  1.360082
           Whitefield_2          85101.32  1.229642

[2221 rows x 2 columns]